In [1]:
from __future__ import print_function, division 
import json                                     
import csv                                      
import numpy as np 
import pandas as pd
from deepmoji.sentence_tokenizer import SentenceTokenizer                                       
from deepmoji.model_def import deepmoji_feature_encoding                                        
from deepmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH 
from sklearn.cross_validation import train_test_split
from keras.layers import Dense
import pickle
from keras.layers import Input, Dense, Dropout
from keras.models import Model

! CUDA_VISIBLE_DEVICES="0"

Using TensorFlow backend.


RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
maxlen = 20                                     
batch_size = 32                                 

print('Tokenizing using dictionary from {}'.format('/app/DeepMoji/model/vocabulary.json'))                                 
with open('/app/DeepMoji/model/vocabulary.json', 'r') as f:                
    vocabulary = json.load(f)     
    
print('Loading model from {}.'.format(PRETRAINED_PATH))                                         
model = deepmoji_feature_encoding(maxlen, '/app/DeepMoji/model/deepmoji_weights.hdf5')                                      
model.summary()    

Tokenizing using dictionary from /app/DeepMoji/model/vocabulary.json
Loading model from /tmp/DeepMoji/model/deepmoji_weights.hdf5.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loading weights for embedding
Loading weights for bi_lstm_0
Loading weights for bi_lstm_1
Loading weights for attlayer
Ignoring weights for softmax
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 256)      12800000    input_1[0][0]                    
_______

In [4]:
TEST_SENTENCES = [u'I love mom\'s cooking',     
                  u'I love how you never reply back..',                                         
                  u'I love cruising with my homies',                                            
                  u'I love messing with yo mind!!',                                             
                  u'I love you and now you\'re just gone..',                                    
                  u'This is shit',              
                  u'This is the shit']  
st = SentenceTokenizer(vocabulary, maxlen)      
tokenized, _, _ = st.tokenize_sentences(TEST_SENTENCES)                                                                      
print(tokenized)
print('Encoding texts..')                       
encoding = model.predict(tokenized)             

print('First 5 dimensions for sentence: {}'.format(TEST_SENTENCES[3]))                          
print(encoding)                          

[[   18    87  3497  1864     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [   18    87    79    13   122   935    85    34     0     0     0     0
      0     0     0     0     0     0     0     0]
 [   18    87 11824    31    41  5113     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [   18    87  3745    31  2324   440    50     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [   18    87    13    12   111   102    42   642    34     0     0     0
      0     0     0     0     0     0     0     0]
 [   22    28   172     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [   22    28    10   172     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]
Encoding texts..
First 5 dimensions for sentence: I love messing with yo mind!!
[[-0.0085292   0.0586118  -0.         ...,  

In [5]:
df = pd.read_csv('../data/train.txt',sep='\t',header=(0), encoding = 'utf8')
df.set_index('id')
df.head()

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


In [6]:
len(df)

30160

In [ ]:
x = []
y = []

for i in range(len(df)):
    tokenized, _, _ = st.tokenize_sentences([df['turn1'][i],df['turn2'][i],df['turn3'][i]]) 
    encoding = model.predict(tokenized) 
    x.append(encoding)
    y.append(df['label'][i])

In [5]:
print(len(x_))
print(len(y_))

30160
30160


In [ ]:
len(x[1][5])

In [34]:
2304/3

768.0

In [14]:
with open('x_test', 'wb') as f:
    pickle.dump(x, f)
with open('y_test', 'wb') as f:
    pickle.dump(y, f)

In [2]:
with open('x_test', 'rb') as f:
    x = pickle.load(f)
with open('y_test', 'rb') as f:
    y = pickle.load(f)

In [3]:
from keras.utils import to_categorical

In [27]:

label2emotion = {0:"others", 1:"happy", 2: "sad", 3:"angry"}
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}

In [5]:
for i in range(len(y)):
    y[i] = emotion2label[y[i]]

In [6]:
y = to_categorical(y)

In [7]:
y[1]

array([ 0.,  0.,  0.,  1.])

In [8]:
for i in range(len(x)):
    x[i] = np.concatenate(x[i], axis=None)

In [9]:
len(x)

30160

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [12]:
print(len(x_train), len(x_test))

20207 9953


In [29]:
inputs = Input(shape=(6912,))
model = Dense(3951, activation='relu')(inputs)
model = Dropout(0.5)(model)
model = Dense(1024, activation='relu')(model)
model = Dropout(0.3)(model)
model = Dense(512, activation='relu')(model)
model = Dropout(0.2)(model)
model = Dense(64, activation='relu')(model)
predictions = Dense(4, activation='softmax')(model)
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(np.array(x_train),
          y_train,
          validation_data=(np.array(x_test),y_test),
          shuffle=True,
          batch_size=64,
          epochs=10)

Train on 20207 samples, validate on 9953 samples
Epoch 1/10
20207/20207 [==============================] - 13s 663us/step - loss: 0.5508 - acc: 0.7996 - val_loss: 0.4260 - val_acc: 0.8536
Epoch 2/10
20207/20207 [==============================] - 12s 591us/step - loss: 0.3876 - acc: 0.8655 - val_loss: 0.3722 - val_acc: 0.8654
Epoch 3/10
20207/20207 [==============================] - 12s 588us/step - loss: 0.3382 - acc: 0.8858 - val_loss: 0.4288 - val_acc: 0.8503
Epoch 4/10
20207/20207 [==============================] - 12s 588us/step - loss: 0.3062 - acc: 0.8963 - val_loss: 0.4804 - val_acc: 0.8682
Epoch 5/10
20207/20207 [==============================] - 12s 591us/step - loss: 0.2731 - acc: 0.9134 - val_loss: 0.4505 - val_acc: 0.8646
Epoch 6/10
20207/20207 [==============================] - 12s 592us/step - loss: 0.2536 - acc: 0.9257 - val_loss: 0.4376 - val_acc: 0.8707
Epoch 7/10
20207/20207 [==============================] - 12s 589us/step - loss: 0.2168 - acc: 0.9347 - val_loss: 0.5

In [16]:
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))

In [31]:
predictions_ = model.predict(np.array(x_test))

In [34]:
getMetrics(predictions_,y_test)

True Positives per class :  [ 4428.  1114.  1553.  1569.]
False Positives per class :  [ 536.  271.  237.  245.]
False Negatives per class :  [ 499.  257.  275.  258.]
Class happy : Precision : 0.804, Recall : 0.813, F1 : 0.808
Class sad : Precision : 0.868, Recall : 0.850, F1 : 0.858
Class angry : Precision : 0.865, Recall : 0.859, F1 : 0.862
Ignoring the Others class, Macro Precision : 0.8456, Macro Recall : 0.8403, Macro F1 : 0.8430
Ignoring the Others class, Micro TP : 4236, FP : 753, FN : 790
Accuracy : 0.8705, Micro Precision : 0.8491, Micro Recall : 0.8428, Micro F1 : 0.8459


(0.8704913091530192,
 0.84906794948887554,
 0.84281734978113809,
 0.84593110334498256)

In [17]:
d_class_weights

{0: 0.50441530639550447,
 1: 1.7770445439547491,
 2: 1.3801940325828299,
 3: 1.3694151834362513}

In [25]:
def getMetrics(predictions, ground):
    """Given predicted labels and the respective ground truth labels, display some metrics
    Input: shape [# of samples, NUM_CLASSES]
        predictions : Model output. Every row has 4 decimal values, with the highest belonging to the predicted class
        ground : Ground truth labels, converted to one-hot encodings. A sample belonging to Happy class will be [0, 1, 0, 0]
    Output:
        accuracy : Average accuracy
        microPrecision : Precision calculated on a micro level. Ref - https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/16001
        microRecall : Recall calculated on a micro level
        microF1 : Harmonic mean of microPrecision and microRecall. Higher value implies better classification  
    """
    # [0.1, 0.3 , 0.2, 0.1] -> [0, 1, 0, 0]
    discretePredictions = to_categorical(predictions.argmax(axis=1))
    
    truePositives = np.sum(discretePredictions*ground, axis=0)
    falsePositives = np.sum(np.clip(discretePredictions - ground, 0, 1), axis=0)
    falseNegatives = np.sum(np.clip(ground-discretePredictions, 0, 1), axis=0)
    
    print("True Positives per class : ", truePositives)
    print("False Positives per class : ", falsePositives)
    print("False Negatives per class : ", falseNegatives)
    
    # ------------- Macro level calculation ---------------
    macroPrecision = 0
    macroRecall = 0
    # We ignore the "Others" class during the calculation of Precision, Recall and F1
    for c in range(1, 4):
        precision = truePositives[c] / (truePositives[c] + falsePositives[c])
        macroPrecision += precision
        recall = truePositives[c] / (truePositives[c] + falseNegatives[c])
        macroRecall += recall
        f1 = ( 2 * recall * precision ) / (precision + recall) if (precision+recall) > 0 else 0
        print("Class %s : Precision : %.3f, Recall : %.3f, F1 : %.3f" % (label2emotion[c], precision, recall, f1))
    
    macroPrecision /= 3
    macroRecall /= 3
    macroF1 = (2 * macroRecall * macroPrecision ) / (macroPrecision + macroRecall) if (macroPrecision+macroRecall) > 0 else 0
    print("Ignoring the Others class, Macro Precision : %.4f, Macro Recall : %.4f, Macro F1 : %.4f" % (macroPrecision, macroRecall, macroF1))   
    
    # ------------- Micro level calculation ---------------
    truePositives = truePositives[1:].sum()
    falsePositives = falsePositives[1:].sum()
    falseNegatives = falseNegatives[1:].sum()    
    
    print("Ignoring the Others class, Micro TP : %d, FP : %d, FN : %d" % (truePositives, falsePositives, falseNegatives))
    
    microPrecision = truePositives / (truePositives + falsePositives)
    microRecall = truePositives / (truePositives + falseNegatives)
    
    microF1 = ( 2 * microRecall * microPrecision ) / (microPrecision + microRecall) if (microPrecision+microRecall) > 0 else 0
    # -----------------------------------------------------
    
    predictions = predictions.argmax(axis=1)
    ground = ground.argmax(axis=1)
    accuracy = np.mean(predictions==ground)
    
    print("Accuracy : %.4f, Micro Precision : %.4f, Micro Recall : %.4f, Micro F1 : %.4f" % (accuracy, microPrecision, microRecall, microF1))
    return accuracy, microPrecision, microRecall, microF1

In [35]:
t[1]

array([ 1.,  0.,  0.,  0.])

In [36]:
predictions_

array([[  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  1.00000000e+00,   2.64081920e-36,   1.88661535e-38,
          1.94478207e-22],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00],
       ..., 
       [  1.35437906e-01,   1.37659383e-03,   4.68777418e-01,
          3.94408107e-01],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00],
       [  9.96649802e-01,   2.25211587e-03,   3.94858507e-04,
          7.03235390e-04]], dtype=float32)